In [23]:
import pandas as pd
import seaborn as sns
from datetime import datetime, date


In [24]:
df_values = pd.read_csv('Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_values.csv')
df_labels = pd.read_csv('Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_labels.csv')
df_values['construction_year'].value_counts()

0       20709
2010     2645
2008     2613
2009     2533
2000     2091
2007     1587
2006     1471
2003     1286
2011     1256
2004     1123
2012     1084
2002     1075
1978     1037
1995     1014
2005     1011
1999      979
1998      966
1990      954
1985      945
1980      811
1996      811
1984      779
1982      744
1994      738
1972      708
1974      676
1997      644
1992      640
1993      608
2001      540
1988      521
1983      488
1975      437
1986      434
1976      414
1970      411
1991      324
1989      316
1987      302
1981      238
1977      202
1979      192
1973      184
2013      176
1971      145
1960      102
1967       88
1963       85
1968       77
1969       59
1964       40
1962       30
1961       21
1965       19
1966       17
Name: construction_year, dtype: int64

In [25]:
df_original = pd.merge(df_values, df_labels, on = 'id', how = 'inner')
df_original.columns

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group', 'status_group'],
      dtype='object')

### Clean data/one hot encoding
this portion will look at all the object data points and see whether or not they should be one hot encoded and/or binned.
1. filter out all the non-object data types and count the number of unique values to see if they would be uselful for classification. 
2. Delete columns that have significant overlap or are not good for one-hot-encoding
3. For date of recording, we plan on turning this into an interger by subtracting every date from the most recent date to get a value for how many dates have passed.

In [26]:
uvdict = {}

for column in df_original.select_dtypes(exclude=['int','float']):
    values_list = df_original[column].unique()
    uvdict[column] = len(values_list)

uvdict

{'date_recorded': 356,
 'funder': 1898,
 'installer': 2146,
 'wpt_name': 37400,
 'basin': 9,
 'subvillage': 19288,
 'region': 21,
 'lga': 125,
 'ward': 2092,
 'public_meeting': 3,
 'recorded_by': 1,
 'scheme_management': 13,
 'scheme_name': 2697,
 'permit': 3,
 'extraction_type': 18,
 'extraction_type_group': 13,
 'extraction_type_class': 7,
 'management': 12,
 'management_group': 5,
 'payment': 7,
 'payment_type': 7,
 'water_quality': 8,
 'quality_group': 6,
 'quantity': 5,
 'quantity_group': 5,
 'source': 10,
 'source_type': 7,
 'source_class': 3,
 'waterpoint_type': 7,
 'waterpoint_type_group': 6,
 'status_group': 3}

In [27]:
df_original.lga.value_counts()

Njombe          2503
Arusha Rural    1252
Moshi Rural     1251
Bariadi         1177
Rungwe          1106
                ... 
Moshi Urban       79
Kigoma Urban      71
Arusha Urban      63
Lindi Urban       21
Nyamagana          1
Name: lga, Length: 125, dtype: int64

In [28]:
# returns the number of nonvalues in the dictionary
nan_dict = {}

for column in df_original.select_dtypes(exclude=['int','float']):
    nan_dict[column] = df_original[column].isna().sum()

nan_dict

{'date_recorded': 0,
 'funder': 3635,
 'installer': 3655,
 'wpt_name': 0,
 'basin': 0,
 'subvillage': 371,
 'region': 0,
 'lga': 0,
 'ward': 0,
 'public_meeting': 3334,
 'recorded_by': 0,
 'scheme_management': 3877,
 'scheme_name': 28166,
 'permit': 3056,
 'extraction_type': 0,
 'extraction_type_group': 0,
 'extraction_type_class': 0,
 'management': 0,
 'management_group': 0,
 'payment': 0,
 'payment_type': 0,
 'water_quality': 0,
 'quality_group': 0,
 'quantity': 0,
 'quantity_group': 0,
 'source': 0,
 'source_type': 0,
 'source_class': 0,
 'waterpoint_type': 0,
 'waterpoint_type_group': 0,
 'status_group': 0}

'recorded_by', 'funder', 'scheme_name' were the three columns to delete outright. Funder was removed because it was nearly identical to installer but contained fewer unique values. For scheme_name, nearly half the data was missing and it seem to relate to scheme management, and 'recorded_by' only has one value making it useless for predictions.

In [29]:
# columns to delete
try:
    df_original.drop(['recorded_by', 'funder', 'scheme_name','subvillage'], axis = 1, inplace = True)
except:
    print('already deleted')


In [30]:
nan_dict = {}

for column in df_original.select_dtypes(exclude=['int','float']):
    nan_dict[column] = df_original[column].isna().sum()

nan_dict

{'date_recorded': 0,
 'installer': 3655,
 'wpt_name': 0,
 'basin': 0,
 'region': 0,
 'lga': 0,
 'ward': 0,
 'public_meeting': 3334,
 'scheme_management': 3877,
 'permit': 3056,
 'extraction_type': 0,
 'extraction_type_group': 0,
 'extraction_type_class': 0,
 'management': 0,
 'management_group': 0,
 'payment': 0,
 'payment_type': 0,
 'water_quality': 0,
 'quality_group': 0,
 'quantity': 0,
 'quantity_group': 0,
 'source': 0,
 'source_type': 0,
 'source_class': 0,
 'waterpoint_type': 0,
 'waterpoint_type_group': 0,
 'status_group': 0}

In [31]:
# replace NaN values with 'not_listed string'
df_original.loc[:, ['installer', 'scheme_management','public_meeting','permit']] = df_original.loc[:, ['installer', 'scheme_management','subvillage','public_meeting','permit']].fillna('Not known')


In [32]:
# replace values with fewer than 200
threshold = 200
cols = ['installer','ward']

for col in cols:
    valued_dict = dict(df_original[col].value_counts())
    safe_values = list(key for key, value in valued_dict.items() if value >= threshold)
#     replace_values = list(filter(lambda x: x not in safe_values, all_values))
    df_original.loc[:, col] = df_original.loc[:, col].map(lambda y: 'other' if y not in safe_values else y)

df_original.installer.value_counts()

other                         19710
DWE                           17402
Not known                      3672
Government                     1825
RWE                            1206
Commu                          1060
DANIDA                         1050
KKKT                            898
Hesawa                          840
0                               777
TCRS                            707
Central government              622
CES                             610
Community                       553
DANID                           552
District Council                551
HESAWA                          539
World vision                    408
LGA                             408
WEDECO                          397
TASAF                           396
District council                392
Gover                           383
AMREF                           329
TWESA                           316
WU                              301
Dmdd                            287
ACRA                        

In [33]:
basedate = datetime(2020, 1, 1)
df_original['days_since'] = df_original.loc[:,'date_recorded'].map(lambda x: (basedate - datetime.strptime(x, "%Y-%m-%d")).days)
df_original['days_since']

0        3215
1        2492
2        2501
3        2529
4        3094
         ... 
59395    2434
59396    3161
59397    3187
59398    3221
59399    3206
Name: days_since, Length: 59400, dtype: int64

In [34]:
basedate = date(2020, 1, 1)
# df_original['time_since_recording'] = df_original['date_recorded'].dt.days

In [35]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 38 columns):
id                       59400 non-null int64
amount_tsh               59400 non-null float64
date_recorded            59400 non-null object
gps_height               59400 non-null int64
installer                59400 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null float64
wpt_name                 59400 non-null object
num_private              59400 non-null int64
basin                    59400 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           59400 non-null object
scheme_management        59400 non-null object
permit                   59400 non-null object
c

Index(['amount_tsh', 'date_recorded', 'funder', 'gps_height', 'installer',
       'longitude', 'latitude', 'wpt_name', 'num_private', 'basin',
       'subvillage', 'region', 'region_code', 'district_code', 'lga', 'ward',
       'population', 'public_meeting', 'recorded_by', 'scheme_management',
       'scheme_name', 'permit', 'construction_year', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group',
       'status_group'],
      dtype='object')

Index(['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private',
       'region_code', 'district_code', 'population', 'construction_year'],
      dtype='object')